# 蛋白质编码

读取AMPs和notAMPs序列，转化为2个通道的特征向量表示。其中通道1的数据来自hmmer的profile；通道2的数据来自氨基酸Onehot编码;通道3来自AA的物化性质

## 导入包

In [17]:
import numpy as np
from Bio import SeqIO
import json
import re

## 全局变量

In [1]:
#jsonfiles = ['./data/benchmark/AMPs_50_hmm_profil.json','./data/benchmark/notAMPs_50_hmm_profil.json']
#fastafiles=['./data/benchmark/wpAMPs.fasta','./data/benchmark/wpnotAMPs.fasta']
#files=['./data/benchmark/AMPs_50.fasta','./data/benchmark/notAMPs_50.fasta']
text='PQRYWTMNVELHSFCIKADG'

## 矩阵归一化

In [2]:
# 对矩阵进行归一化
def maxminnorm(array):
    maxcols=array.max(axis=0)
    mincols=array.min(axis=0)
    data_shape = array.shape
    data_rows = data_shape[0]
    data_cols = data_shape[1]
    t=np.empty((data_rows,data_cols))
    for i in range(data_cols):
        if maxcols[i] > mincols[i]:
            t[:,i]=(array[:,i]-mincols[i])/(maxcols[i]-mincols[i])
    return t

## load_hmm_prof: 读入序列的hmmer profil 文件

In [16]:
# 加载来自hmmer profil的数据
# jsonfile 存储hmmer profil数据的json格式文件名
# fastafile 序列的参照文件，fasta格式
# numAA>0 表示从头取numAA个氨基酸的profil，尾不足补0; numAA<0 表示从尾往前取numAA个氨基酸的profil，头不足补0
def load_hmm_prof(jsonfile, fastafile, numAA=50):
    records = SeqIO.parse(fastafile, 'fasta')
    seqID = [str(x.id) for x in records]
    records.close()
    
    M = len(seqID)
    N = abs(numAA) * 20
    
    X = np.ndarray((M,N))

    k = 0
    
    fr = open(jsonfile,'r')
    p = json.load(fr)
    fr.close()
    
    for key in seqID:
        ary = p[key]
        tm = np.array(ary).reshape([-1,20])
        tm = tm[1:,:]
        c = len(ary)-20
        if numAA > 0:
            if c < N:
                tm = maxminnorm(tm)# 归一化
                X[k][:c] = tm.reshape(c)
                X[k][c:] = 0
            elif c == N:
                tm = maxminnorm(tm)# 归一化
                X[k] = tm.reshape(c)
            else:
                t = tm[:numAA,:]
                t = maxminnorm(t)# 归一化
                X[k] = t.reshape(N)
        else:# numAA < 0
            if c < N:
                tm = maxminnorm(tm)
                X[k][-c:] = tm.reshape(c)
                X[k][:-c] = 0
            elif c==N:
                tm = maxminnorm(tm)
                X[k] = tm.reshape(c)
            else:
                t = tm[numAA:,:]
                t = maxminnorm(t)
                X[k] = t.reshape(N)

        k += 1
    return X

## 氨基酸OneHot编码

In [4]:
# numAA=0 蛋白质序列全肽链； numAA>0 从头截止到第numAA个氨基酸； numAA<0 从尾部往前numAA个氨基酸
# 返回一个列表，列表的每一行表示一个蛋白质序列的OneHot编码
def AAOneHot(fastafile, numAA=50):
    X = []
    for seq_record in SeqIO.parse(fastafile, 'fasta'):
            seq = str(seq_record.seq)
            seq = re.sub('[XZUB]',"",seq)
            c = len(seq)
            m = np.zeros([c,20])
            for i in range(c):
                j = text.index(seq[i])
                m[i][j] = 1.0
            m = m.reshape([1,-1])
            X.append(m)
            
    if numAA == 0:
        return X
    elif numAA> 0:
        L = numAA*20
        A = []
        for m in X:
            c = m.size()
            t = []
            if c < L:
                t[:c] = m
                t[c:] = np.zeros(L-c)
            elif c == L:
                t = m
            else:
                t = m[:L]
            A.append(t)
        return A
    else: # numAA < 0
        L = abs(numAA)*20
        A = []
        for m in X:
            c = m.size()
            t = []
            if c < L:
                t[-c:] = m
                t[:-c] = np.zeros(L-c)
            elif c == L:
                t = m
            else:
                t = m[-L:]
            A.append(t)
        return A

## dAAOneHot(): 氨基酸两联体编码

In [52]:
# 两联体编码
def dAAOneHot(fastafile):
    daa=[x+y for x in text for y in text]
    X = []
    for seq_record in SeqIO.parse(fastafile, 'fasta'):
        seq = str(seq_record.seq)
        seq = re.sub('[XZUB]',"",seq)
        t = np.zeros(400)
        for j in range(400):
            t[j] = seq.count(daa[j])
        maxv, minv = t.max(),t.min()
        for j in range(400):
            t[j] = (t[j]-minv)/(maxv-minv)
        X.append(t)

    return X

## 氨基酸物化属性OneHot编码

In [6]:
# numAA=0 蛋白质序列全肽链； numAA>0 从头截止到第numAA个氨基酸； numAA<0 从尾部往前numAA个氨基酸
# 返回一个列表，列表的每一行表示一个蛋白质序列的物化属性的OneHot编码
def AAPhyChemOneHot(fastafile, numAA):
    phychemDict={}
    phychemDict["alcohol"]=("S","T")# 有乙醇基
    phychemDict["aliphatic"]=("I","L","V")# 脂肪族
    phychemDict["aromatic"]=("F","H","W","Y")# 芳香族
    phychemDict["charged"]=("D","E","H","K","R")# 带电性
    phychemDict["positive"]=("K","H","R")# 带正电
    phychemDict["negative"]=("D","E")# 带负电
    phychemDict["polar"]=("A","L","I","P","F","W","M")# 非极性
    phychemDict["small"]=("A","C","D","G","N","P","S","T","V")# 小分子
    phychemDict["turnlike"]=("A","C","D","E","G","H","K","N","Q","R","S","T")
    phychemDict["hydrophobic"]=("A","F","I","L","M","P","V","W","Y")# 疏水
    phychemDict["asa"]=("A","N","D","C","P","S","T","G","V")# 可溶解表面积低于平均值
    phychemDict["pr"]=("F","Y","W")# 在紫外区有光吸收能力
       
    X = []
    keys = phychemDict.keys()
    lskey = list(keys)    
    N = len(keys)
    
    for seq_record in SeqIO.parse(fastafile, 'fasta'):
        seq = str(seq_record.seq)
        seq = re.sub('[XZUB]',"",seq)
        c = len(seq)
        m = np.zeros((len(seq),N))
        
        for i in range(c):
            for j in range(N):
                key = lskey[j]
                val = phychemDict[key]
                if seq[i] in val:
                    m[i][j] = 1
 
        m = m.reshape((1,-1))
        X.append(m)
        
    if numAA == 0:
        return X
    elif numAA > 0:# 只截取蛋白质序列前numAA个氨基酸，不足在尾部补0
        A = []
        L = numAA*20
        for m in X:
            c = m.size()
            t = []
            if c < L:
                t[:c] = m
                t[c:] = np.zeros(L-c)
            elif c == L:
                t = m
            else:
                t = m[:L] 
            A.append(t)
        return A
    else: # numAA<0 截取蛋白质序列后numAA个氨基酸，不足在头部补0
        A = []
        L = abs(numAA)*20
        for m in X:
            c = m.size()
            t = []
            if c < L:
                t[-c:] = m
                t[:-c] = np.zeros(L-c)
            elif c == L:
                t = m
            else:
                t = m[-L:]
            A.aapend(t)
        return X

# 构建卷积网络进行训练

## 导入tensorflow的包

In [7]:
import tflearn
import tensorflow as tf
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.metrics import accuracy_score, auc, roc_curve, matthews_corrcoef

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


## 构建Alex网络结构

In [9]:
def create_alexnet(num_classes,channels=1):
    # Building 'AlexNet'
    network = input_data(shape=[None, 50, 20, channels])
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)#[-1,10,10,96]
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)#[-1,5,5,256]
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)#[-1,3,3,256]
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, num_classes, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    return network

## 构建Cifar网络结构

In [10]:
def create_cifarnet(num_classes,channels=1):
    # Real-time data preprocessing
    img_prep = ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()
    
    # Real-time data augmentation
    img_aug = ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_rotation(max_angle=25.)
    
    # Convolutional network building
    network = input_data(shape=[None, 20, 20, channels],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
    network = conv_2d(network, 32, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = dropout(network, 0.75)
    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = dropout(network, 0.5)
    network = fully_connected(network, 512, activation='tanh')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='tanh')
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    return network
    

## 构建vgg网络结构

In [11]:
# Building 'VGG Network'
def create_vggnet(num_classes,channels=1):
    network = input_data(shape=[None, 100, 20, channels])

    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)#[-1,25,10,64]

    network = conv_2d(network, 128, 3, activation='relu')
    network = conv_2d(network, 128, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)#[-1,13,5,128]

    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)#[-1,7,3,256]
    
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)
    
    network = fully_connected(network, 2048, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, num_classes, activation='softmax')

    network = regression(network, optimizer='rmsprop',
                         loss='softmax_categorical_crossentropy',
                         learning_rate=0.0001)
    return network

# 定义交叉验证函数

## jack knife测试函数

In [62]:
# 返回预测的结果，预测的结果为各个类的概率
def jackknife_test(X, y, num_classes=2,channels=1,n_epoch=100):
    M = X.shape[0]
    y_pred = np.zeros([M,2])
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(X):
        print("\r In predicting {}".format(test_index))
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        tf.reset_default_graph()
        net = create_alexnet(num_classes,channels)
        model = tflearn.DNN(net, tensorboard_verbose=0)
        model.fit(X_train, y_train, n_epoch, shuffle=True, 
              validation_set=(X_test,y_test),
              show_metric=True, batch_size=32, 
              run_id='AMP_cnn')
        y_pred[test_index] = model.predict(X_test)
        
    return y_pred

## K-fold折叠验证

In [13]:
# 返回预测的结果，预测的结果为各个类的概率
def cross_validate(X,y,n_splits=3,num_classes=2,channels=1,n_epoch=100):
    M = X.shape[0]
    pred_prob = np.zeros([M,2])
    kf = KFold(n_splits)
    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        tf.reset_default_graph()
        net = create_alexnet(num_classes,channels)
        model = tflearn.DNN(net, tensorboard_verbose=0)
        model.fit(X_train, y_train, n_epoch, shuffle=True, 
              validation_set=(X_test,y_test),
              show_metric=True, batch_size=32)
        pred_prob[test_index] = model.predict(X_test)
        
    return pred_prob

## 计算预测性能（指标：ACC、AUC和MCC）

In [14]:
# 计算准确率acc和可接受曲线下面积AUC
from sklearn.metrics import matthews_corrcoef
def metric(y, predprob):
    d1 = len(y)
    y_pred = np.zeros((d1,2))
    for i in range(d1):
        if predprob[i][0] > predprob[i][1]:
            y_pred[i][0] = 1
        else:
            y_pred[i][1] = 1
            
    accuracy=accuracy_score(y[:,0],y_pred[:,0])
    print("accuracy={}".format(accuracy))
    fpr,tpr,thresholds=roc_curve(y[:,0],predprob[:,0],pos_label=1)
    print("AUC={}".format(auc(fpr,tpr)))
    mcc = matthews_corrcoef(y[:,0],y_pred[:,0])
    print("mcc={}".format(mcc))

# 第一层预测器

## 使用iAMP-2L的数据训练及预测
<p/>
<font size=4>数据由王普构建，包含879个AMP和2405个非抗菌他</font>

### 构建训练集和测试集

In [55]:
jsonfiles_wp = ['./data/benchmark/wpAMPs_hmm_profil.json','./data/benchmark/wpnotAMPs_hmm_profil.json']
files_wp=['./data/benchmark/wpAMPs.fasta','./data/benchmark/wpnotAMPs.fasta']
N = 20
X1_1 = load_hmm_prof(jsonfiles_wp[0],files_wp[0],N)
X1_2 = load_hmm_prof(jsonfiles_wp[1],files_wp[1],N)
X1 = np.vstack((X1_1, X1_2)).reshape([-1,N,20])

X2_1 = load_hmm_prof(jsonfiles_wp[0],files_wp[0],-N)
X2_2 = load_hmm_prof(jsonfiles_wp[1],files_wp[1],-N)
X2 = np.vstack((X2_1, X2_2)).reshape([-1,N,20])

X3_1 = np.array(dAAOneHot(files_wp[0]))
X3_2 = np.array(dAAOneHot(files_wp[1]))
X3 = np.vstack((X3_1, X3_2)).reshape([-1,20,20])

m1 = X1_1.shape[0]
m2 = X1_2.shape[0]

X = np.ndarray([m1+m2,20,20,3])
X[:,:,:,0] = X1
X[:,:,:,1] = X2
X[:,:,:,2] = X3

y = np.zeros(m1+m2)
y[m1:] = 1
y = to_categorical(y,2)

### jack-knife 测试

In [ ]:
X,y = shuffle(X,y)
predprob = jackknife_test(X,y,2,3,35)
metric(y,predprob)

Training Step: 321  | total loss: 0.56687 | time: 7.396s
| Momentum | epoch: 004 | loss: 0.56687 - acc: 0.7293 -- iter: 0384/3283


## 使用自己构建的数据集训练和测试
<p></p>
<font size=4>包含800个AMP和800个非AMP</font>

# 第二层预测器

# 其它

## 仅hmmer profil一个通道数据的训练

In [ ]:
X1,y = load_hmm_prof()
X1 = X1.reshape([-1,50,20,1])
y = np.zeros((M,2))
for i in range(M1):
    y[i][0] = 1
for i in range(M1,M):
    y[i][1] = 1
# 交叉验证
X1,y = shuffle(X1,y)
predprob = cross_validate(X1,y,n_splits=5,num_classes=2,channels=1,n_epoch=50)
metric(y, predprob)# print: 0.795, 0.868684

## 两个通道数据的训练

In [ ]:
# 获取数据
X,y = getTwoChannelsArray()
yy = np.zeros((M,2))
for i in range(M1):
    yy[i][0] = 1
for i in range(M1,M):
    yy[i][1] = 1
# 交叉验证
X,y = shuffle(X,yy)
predprob = cross_validate(X,y,n_splits=5,num_classes=2,channels=2,n_epoch=50)
metric(y, predprob) # print: 0.81375, 0.883389


<font size=5> <b>预测结果</b></font><br>
5-fold, cifanet,30 epochs, 879amps+2405 not amps, acc=0.8562, auc=0.9045, mcc=0.6470<br>


## 三个通道数据训练

In [ ]:
X,y = getThreeChannelsArray()
yy = np.zeros((M,2))
for i in range(M1):
    yy[i][0] = 1
for i in range(M1,M):
    yy[i][1] = 1
# 交叉验证
X,y = shuffle(X,yy)
predprob = cross_validate(X,y,n_splits=5,num_classes=2,channels=3,n_epoch=30)
metric(y, predprob)# accuracy=0.820625 AUC=0.9052437499999999

In [ ]:
pred_prob = np.zeros(1600)

In [ ]:
X_train,X_test=X[:1280],X[1280:]
y_train,y_test=y[:1280],y[1280:]
tf.reset_default_graph()
net = create_vggnet(2,3)
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X_train, y_train, n_epoch=30, shuffle=True, 
      show_metric=True, batch_size=32)
predval = model.predict(X_test)
metric(y_test, predval)

In [ ]:
predval = model.predict(X_test)
metric(y_test, predval)

<font size=4><b>预测结果</b></font><br>


## 两联体-前20hmmrof-后20hmmprof,三个通道数据

In [ ]:
# 构建数据集
X1,y1 = load_hmm_prof(20,0)
X2,y2 = load_hmm_prof(20,-1)
X3,y3 = dAAOneHot()
X=np.ndarray([M,20,20,3])
X11 = X1.reshape([M,20,20])
X21 = X2.reshape([M,20,20])
X31 = X3.reshape([M,20,20])
X[:,:,:,0]=X11
X[:,:,:,1]=X21
X[:,:,:,2]=X31
yy = np.zeros((M,2))
for i in range(M1):
    yy[i][0] = 1
for i in range(M1,M):
    yy[i][1] = 1

In [14]:
X,y = shuffle(X,yy)
predprob = cross_validate(X,y,n_splits=5,num_classes=2,channels=3,n_epoch=40)
metric(y, predprob)# 王普的数据，用alexnet, acc=0.90956 auc=0.957706  mcc=0.765609

Training Step: 3319  | total loss: 0.00074 | time: 55.461s
| Momentum | epoch: 040 | loss: 0.00074 - acc: 1.0000 -- iter: 2624/2628
Training Step: 3320  | total loss: 0.00075 | time: 58.234s
| Momentum | epoch: 040 | loss: 0.00075 - acc: 1.0000 | val_loss: 0.58507 - val_acc: 0.8994 -- iter: 2628/2628
--
accuracy=0.9095615103532277
AUC=0.9577068536112904
mcc=0.7656093650437452


# 仅测试代码用

In [59]:
X,y = shuffle(X,y)
X_train,X_test=X[:-656],X[-656:]
y_train,y_test=y[:-656],y[-656:]
tf.reset_default_graph()
net = create_vggnet(2,3)
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X_train, y_train, 30, shuffle=True, 
      validation_set=(X_test,y_test),
      show_metric=True, batch_size=32)
pred_prob= model.predict(X_test)
metric(y_test, pred_prob)

Training Step: 2904  | total loss: 0.00190 | time: 46.510s
| Momentum | epoch: 035 | loss: 0.00190 - acc: 1.0000 -- iter: 2624/2628
Training Step: 2905  | total loss: 0.00214 | time: 49.098s
| Momentum | epoch: 035 | loss: 0.00214 - acc: 1.0000 | val_loss: 0.65024 - val_acc: 0.8826 -- iter: 2628/2628
--
accuracy=0.8826219512195121
AUC=0.9303817483623221
mcc=0.7079099904465954


In [58]:
pred_prob

array([[0.41271517, 0.58728486],
       [0.9371414 , 0.06285849],
       [0.00776255, 0.9922375 ],
       ...,
       [0.08722904, 0.912771  ],
       [0.5964559 , 0.40354416],
       [0.00463584, 0.9953642 ]], dtype=float32)